## 1. 파이썬과 MYSQL 활용
## 모듈
    1)mysqlclient, PyMySQL
    2)pip install mysqlclient, pip install PyMySQL

# mysqlclient 모듈 사용

In [3]:
import MySQLdb


In [10]:
# mysql 서버에 접속
#conn= MySQLdb.connect(host = '127.0.0.1',user = 'root',password='1111',database='testdb')
# dict변수로 담아보기
config = {"host":"127.0.0.1","user":"root","password":"1111","database":"testdb"}
conn = MySQLdb.connect(**config)
#매개변수가 dict일때 
print(conn)
conn.close()

<_mysql.connection open to '127.0.0.1' at 00000218D4E9C4D8>


In [17]:
conn = MySQLdb.connect(**config)
sql = "SELECT empno, ename, sal, job, deptno FROM emp"
cur = conn.cursor() #커서객체
cur.execute(sql)
print(type(cur))
#커서객체 안의 데이터 확인
#for data in cur:
#    print(data)

#unpacking
#튜플로 묶여있는걸 풀어주는걸 언패킹
#for data in cur:
#    print("%d,%s,%d,%s,%d"%data)

#언패킹의 또다른 방법
#for empno, ename, sal, job,deptno in cur:
#    print(empno, ename, sal, job, deptno)

#튜플데이터들을 리스트로 끌어와서 작업해도됨
print(cur)
conn.close()

<class 'MySQLdb.cursors.Cursor'>
7369 SMITH 800.0 CLERK 20
7499 ALLEN 1600.0 SALESMAN 30
7521 WARD 1250.0 SALESMAN 30
7566 JONES 2975.0 MANAGER 20
7654 MARTIN 1250.0 SALESMAN 30
7698 BLAKE 2850.0 MANAGER 30
7782 CLARK 2450.0 MANAGER 10
7788 SCOTT 3000.0 ANALYST 20
7839 KING 5000.0 PRESIDENT 10
7844 TURNER 1500.0 SALESMAN 30
7876 ADAMS 1100.0 CLERK 20
7900 JAMES 950.0 CLERK 30
7902 FORD 3000.0 ANALYST 20
7934 MILLER 1300.0 CLERK 10


In [18]:
#INSERT 

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = """INSERT INTO emp(empno,ename, sal, job, mgr, deptno,hiredate)
VALUES(1,'tom',700,'clerk',7788, 20, now())"""

cur.execute(sql)
conn.commit() 
#commit은 conn 객체가 하는거임
# dml에서는 SELECT말고 commit으로 완료를 해줘야 작업이끝남
conn.close()

In [24]:
conn = MySQLdb.connect(**config)
cur = conn.cursor()

#외부로부터 가져왔다고 가정
v_empno=5
v_ename = 'Johnson'
v_sal = 1500

# v_empno 를 string으로 바꿔줘야됨
#보안성 문제  명령어와 변수를 묶으면 안됨 
#고정된 데이터 따로 변수따로 보내야됨
sql = "INSERT INTO emp(empno, ename, sal, hiredate) VALUES(" + str(v_empno) + \
    ",'" + v_ename + "'," + str(v_sal) + ", now())"

print(sql)

cur.execute(sql)
conn.commit()
conn.close()

INSERT INTO emp(empno, ename, sal, hiredate) VALUES(5,'Johnson',1500, now())


In [28]:
conn = MySQLdb.connect(**config)
cur = conn.cursor()

#외부로부터 가져왔다고 가정
v_empno=10
v_ename = 'Johnson'
v_sal = 1500

# v_empno 를 string으로 바꿔줘야됨
#보안성 문제  명령어와 변수를 묶으면 안됨 
sql = "INSERT INTO emp(empno, ename, sal, hiredate) VALUES(%s, %s, %s, now())"
sql_data = (str(v_empno),v_ename,str(v_sal))


cur.execute(sql, sql_data)
conn.commit()
conn.close()

In [40]:
#delete
conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "delete from emp where empno = 1 or empno = 5;"

cur.execute(sql)
conn.commit()
conn.close()

# PyMySQL 모듈 사용

In [1]:
import pymysql


In [32]:
conn = pymysql.connect(host = '127.0.0.1',user = 'root',passwd='1111',db='testdb',port = 3306,charset='utf8')

print(conn)
conn.close()


In [2]:
config = {"host":"127.0.0.1","user":"root","passwd":"1111","db":"testdb","port":3306,"charset":"utf8"}

conn = pymysql.connect(**config)
print(conn)
conn.close()

In [38]:
conn = pymysql.connect(**config)
cur = conn.cursor()
## unsigned 양수만
sql = """
    CREATE TABLE IF NOT EXISTS pythonTable(
    id int unsigned not null auto_increment,
    name varchar(20) not null,
    regdate datetime,
    primary key(id)
    );
"""
cur.execute(sql)
cur.execute("show tables")

for i in cur:
    print(i)
conn.commit()

conn.close()

('bonus',)
('dept',)
('emp',)
('pythontable',)
('salgrade',)
('tbla',)
('tblb',)
('tblbook',)
('tblc',)
('tblindex',)
('tbltest',)
('vw_emp1',)


In [41]:
conn = pymysql.connect(**config)
try:
    cur = conn.cursor()
    sql = "INSERT INTO pythontable(name, regdate) VALUES('tom',now());"
    for num in range(10):
        cur.execute(sql)
    
    conn.commit()
    
finally:
    conn.close()

In [13]:
#CSV 파일을 DB에 저장하기 
"""
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);
"""
from datetime import datetime
import csv
conn = pymysql.connect(**config)
cur = conn.cursor()
sql = """
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);
"""
cur.execute(sql)

try:
    
    #csv 모듈의 함수 open 함수- reader, readline 한줄이나 한번에 읽는건대 csv.reader는 콤마를 읽을 수 있음
    file = csv.reader(open("input.csv"))
    print(file)
    #print(next(file))
    header = next(file)
    # db의 Purchase Date는  날짜형식으로 전달해야함 string말고
    # 필드의 개수를 알기위해 header 변수 활용
    for row in file:
        data = []
        for col in range(len(header)):
            # 0~ 3필드까지는 그냥 넣고 4번째 필드 date필드에서는 형변환해야함
            #Y 4글자년도 y2글자 년도
            if col == 4:
                tmp = datetime.strptime(row[col],"%Y-%m-%d")
                data.append(tmp)
            else:
                data.append(row[col])
            # Strptime Strftime 
        print(data)
        cur.execute("INSERT INTO suppliers VALUES(%s,%s,%s,%s,%s)",data)
    
    conn.commit()
    
finally:
    conn.close()

['A', '001-1001', '2341', '500000', datetime.datetime(2014, 1, 20, 0, 0)]
['A', '001-1002', '2341', '500000', datetime.datetime(2014, 1, 20, 0, 0)]
['A', '001-1003', '5467', '750000', datetime.datetime(2014, 1, 20, 0, 0)]
['A', '001-1004', '5467', '750000', datetime.datetime(2014, 1, 20, 0, 0)]
['B', '001-1005', '7009', '250000', datetime.datetime(2018, 1, 30, 0, 0)]
['B', '001-1006', '7009', '250000', datetime.datetime(2018, 1, 30, 0, 0)]
['B', '001-1007', '6650', '125000', datetime.datetime(2017, 2, 3, 0, 0)]
['B', '001-1008', '6650', '125000', datetime.datetime(2017, 2, 3, 0, 0)]
['C', '001-1009', '3321', '615000', datetime.datetime(2017, 2, 3, 0, 0)]
['C', '001-1010', '3321', '615000', datetime.datetime(2017, 2, 10, 0, 0)]
['C', '001-1011', '3321', '615000', datetime.datetime(2017, 2, 17, 0, 0)]
['C', '001-1012', '3321', '615000', datetime.datetime(2017, 2, 24, 0, 0)]


In [14]:
conn = pymysql.connect(**config)
cur = conn.cursor()
sql = """
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);
"""
cur.execute(sql)

try:
    
    #csv 모듈의 함수 open 함수- reader, readline 한줄이나 한번에 읽는건대 csv.reader는 콤마를 읽을 수 있음
    file = csv.reader(open("input.csv"))
    #print(file)
    #print(next(file))
    header = next(file)
    sql = "delete from suppliers"
    cur.execute(sql)
    conn.commit()
    for row in file:
        cur.execute("INSERT INTO suppliers VALUES(%s,%s,%s,%s,%s)",row)
    
    # db의 Purchase Date는  날짜형식으로 전달해야함 string말고
    # 필드의 개수를 알기위해 header 변수 활용
  #  for row in file:
  #      data = []
  #      for col in range(len(header)):
            # 0~ 3필드까지는 그냥 넣고 4번째 필드 date필드에서는 형변환해야함
            #Y 4글자년도 y2글자 년도
  #          if col == 4:
  #              tmp = datetime.strptime(row[col],"%Y-%m-%d")
  #              data.append(tmp)
  #          else:
  #              data.append(row[col])
  #          # Strptime Strftime 
  #      print(data)
   #     cur.execute("INSERT INTO suppliers VALUES(%s,%s,%s,%s,%s)",data)
    
    conn.commit()
    
finally:
    conn.close()

C:\Users\acorn\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1050, "Table 'suppliers' already exists")
  result = self._query(query)


# db에서 읽어서 파일 읽어서 저장

In [20]:
conn = pymysql.connect(**config)
try:
    cur = conn.cursor()
    
    cur.execute("SELECT * FROM suppliers WHERE cost>500000")
    # for i in cur : print(i)           # sql로 데이터들 확인
    file = csv.writer(open("output.csv","w"),delimiter=",")
    header = ["Supplier Name","Invoice Name","Part Number","Cost","Purchase Date"]
    file.writerow(header)
    
    for row in cur:
        file.writerow(row)
finally:
    conn.close()

# 2. 파이썬과 SQLite
    1. 로컬 DB ( 네트워크 기능이 없음 하나의 컴퓨터안에서만 할수있는)
     로컬 DB 제품이 많음 - Access, -SQLite, -Microsoft SQL Server Compact Edition
     -Firebird Embeded Server -H2 -HSQL -Apache Derby
    2. 특징
        1) 파일기반의 DBMS , 저메모리, 빠른 처리 속도
        2) 오픈 소스 
        3) 별도의 DB서버가 없어도 쉽고 편리하게 사용할 수 있는 Embeded SQL DB엔진
        4) 안드로이드, 아이폰 등의 스마트폰에 기본 내장된 DB
        5) 표준 SQL 지원
        6) 파이썬에 내장
        ---------------
        7) 칼럼을 삭제하거나 변경하는 것 등이 제한
        8) Left Outer Join만 지원 
        9) 읽기 전용 뷰만 제공 
    3. 공식 사이트 : http://sqlite.org
    4. 클라이언트 툴 
        - http://www.sqliteexpert.com

In [21]:
# sqlitexpert를 이용해서 db생성 : mysqlite.db
"""
CREATE TABLE tblmember(
    id int,
    name varchar(10),
    regdate datetime
);
"""

"""
INSERT INTO tblmember VALUES(1,'홍길동','2020-5-5')
INSERT INTO tblmember VALUES(1,'임꺽정',date('now'))
"""

"\nINSERT INTO tblmember VALUES(1,'홍길동','2020-5-5')\nINSERT INTO tblmember VALUES(1,'임꺽정',date('now'))\n"

In [23]:
import sqlite3

print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.31.1


In [29]:
conn = sqlite3.connect("mysqlite.db")
cur = conn.cursor()

cur.execute("INSERT INTO tblmember VALUES(2,'루루라','2020-5-4')")
cur.execute("SELECT * FROM tblmember")
for row in cur:
    print(row)
conn.commit()
conn.close()

(1, '홍길동', '2020-5-5')
(1, '임꺽정', '2020-05-28')
(2, '루루라', '2020-5-4')


# csv 파일 읽어서 sqlite에 넣기

In [ ]:
import csv
file = csv.reader(open("input.csv"),delimiter =',')
header = next(file)

conn = sqlite3.connect("mysqlite.db")
cur = conn.cursor()
sql =  """
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);
"""
cur.execute(sql)
for row in file:
    cur.execute("INSERT INTO suppliers VALUES(?,?,?,?,?)",row)

for row in cur:
    print(row)
conn.commit()
conn.close()

# 메모리 DB

# Text 타입 (글을 많이쓸떄) 

In [40]:

conn=sqlite3.connect(":memory:")
cur = conn.cursor()
#print(cur)
cur.execute("CREATE TABLE phonebook(name text,phonenum varchar(20));")
cur.execute("INSERT INTO phonebook VALUES(?,?);",("tom","010-222-222"))

In [42]:
cur.execute("select * from phonebook")
#fetchmany(n) n번 까지
for i in cur:
    print(i)
    
for i in cur.fetchmany(3):
    print(i)

('tom', '010-222-222')


# 입력데이터 인자를 변수명 처리


In [45]:
sql = "INSERT INTO phonebook VALUES(:vName,:vPhone)"
cur.execute(sql,{"vPhone":"010-222-3333","vName":"Johnson"})

In [46]:
#여러 개의 데이터를 한번에 처리
#executemany() 함수 자동으로 처리
datalist = (('jane','010-333-3333'),('sam','010-233-3333'),('paul','010-433-3333'))
cur.executemany("INSERT INTO phonebook VALUES(?,?)",datalist)

In [48]:
# 작업한 내용을 스크립트로 저장
f = open("data/dump.sql","w")
for line in conn.iterdump():
    #print(line)
    f.write("{}\n".format(line))
    
f.close()
conn.close()

In [52]:
# 스크립트를 저장해야함
conn=sqlite3.connect(":memory:")
cur = conn.cursor()

f = open("data/dump.sql")
script = f.read()
#print(script)

cur.executescript(script)

cur.execute("select * from phonebook")
for i in cur.fetchmany(3):
    print(i)

f.close()
conn.close()

('tom', '010-222-222')
('Johnson', '010-222-3333')
('jane', '010-333-3333')
